In [128]:
import math
from collections import Counter
import pickle
import re
import nltk
from nltk.corpus import stopwords
from lyricsgenius import Genius

with open('../lyrics-updated.pkl', 'rb') as f:
    song_lyrics = pickle.load(f)


with open('../classifications.pkl', 'rb') as f1:
    classifications = pickle.load(f1)

with open('../output/big-songs-updated.pkl', 'rb') as f2:
    big_songs = pickle.load(f2)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def tokenize(lyrics):
    words = lyrics.lower().split()
    return [word for word in words if word not in stop_words]

k1 = 1.0
b = 0.9

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\civip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [129]:
# call the api to get the lyrics

genius = Genius('HhcPCGMwENzdfMQlauEV1Om0UormkWc6Wrwxri-LFTfBSNkzhwMSVUnR5tZG8eXW')
genius.remove_section_headers = True
genius.skip_non_songs = False
genius.excluded_terms = ["Remix", "Live"]

# song_name = "Happy"
# artist = "Pharrel Williams"
# song_name = "Firework"
# artist = "Katy Perry"
# song_name = "I Hate Everything About You"
# artist = "Three Days Grace"
# song_name = "Coffee"
# artist = "Chappell Roan"
song_name = "Tears in Heaven"
artist = "Eric Clapton"


song = genius.search_song(song_name, artist)
pos = song.lyrics.find(f'{song_name} Lyrics')
query = song.lyrics[pos:]



Searching for "Tears in Heaven" by Eric Clapton...
Done.


In [131]:
# tokenize all documents
tokenized_songs = {}
for category, all_lyrics in big_songs.items():
    tokenized_songs[category] = tokenize(all_lyrics)

# calculate document lengths
doc_lengths = {}
for  category, all_lyrics in tokenized_songs.items():
    doc_lengths[category] = len(all_lyrics)

# avg doc length
avg_doc_length = sum(doc_lengths.values()) / len(doc_lengths)


In [132]:
# calculate doc freqs
doc_freqs = {}
for doc in tokenized_songs.values():
    unique_terms = set(doc)
    for term in unique_terms:
        doc_freqs[term] = doc_freqs.get(term, 0) + 1

In [134]:
scores = []
for doc_id, doc in tokenized_songs.items():
    score = bm25(query, doc, doc_lengths[doc_id])
    scores.append((doc_id, score))

# Sort documents by score in descending order
ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)

# Display the ranking results
print("Document Rankings:")
for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
    print(f"Rank {rank}: Document {doc_id} with score {score:.4f}")